In [1]:
import numpy as np
import scipy as sp
import scipy.io.wavfile as spw
import time
import wave
import struct
import math

# Part 1

In [2]:
#Class for DTMF Decoder
class dtmfdecoder:
    #Create Goertzel Parameters
    def __init__(self, sampr):
        self.sampr = sampr
        #Establish known frequencies that are given to us
        self.g_fr = [1209,1336,1477,1633,697,770,852,941]
        #Setup individual parameters so we can sample the file all at once
        self.sr1 = {}
        self.sr2 = {}
        self.power = {}
        self.N = {}
        self.K = {}
        
        for i in self.g_fr:
            self.sr1[i] = 0
            self.sr2[i] = 0
            self.power[i] = 0
            self.N[i] = 0
            normfreq = i/sampr
            self.K[i] = 2*np.cos(2*np.pi*normfreq)
            
            
            
    #Program to generate the number
    def number(self, freq):
        
        #iterated to make the code robust for part 2 of the problem
        side = [697,770,852,941]
        bottom = [1209,1336,1477,1633]

        
        #get side freq
        sidefreq = 0
        sidefreq_v = 0
        #compare second and first harmonics
        for i in side:
            if freq[i] > sidefreq_v:
                sidefreq_v = freq[i]
                sidefreq = i
                
        #get bottom freq
        botfreq = 0
        botfreq_v = 0
        botfreq_v2 = 0
        for i in bottom:
            if freq[i] > botfreq_v:
                botfreq_v = freq[i]
                botfreq = i
        
        #assign number to frequecy going across the grids one by one
        if sidefreq==697.0:
            if botfreq==1209.0:
                return "1"
            elif botfreq==1336.0:
                return "2"
            elif botfreq==1477.0:
                return "3"
            elif botfreq==1633.0:
                return "A"
        elif sidefreq==770.0:
            if botfreq==1209.0:
                return "4"
            elif botfreq==1336.0:
                return "5"
            elif botfreq==1477.0:
                return "6"
            elif botfreq==1633.0:
                return "B"
        elif sidefreq==852.0:
            if botfreq==1209.0:
                return "7"
            elif botfreq==1336.0:
                return "8"
            elif botfreq==1477.0:
                return "9"
            elif botfreq==1633.0:
                return "C"
        elif sidefreq==941.0:
            if botfreq==1209.0:
                return "*"
            elif botfreq==1336.0:
                return "0"
            elif botfreq==1477.0:
                return "#"
            elif botfreq==1633.0:
                return "D"
    #Run the calculation for the DTMF
    def calc(self,samp):
        freqs = {}
        for f in self.g_fr:
            s = samp + (self.K[f] * self.sr1[f]) - self.sr2[f]
            self.sr2[f] = self.sr1[f]
            self.sr1[f] = s
            self.N[f] += 1
            power = (self.sr2[f]*self.sr2[f]) + (self.sr1[f]*self.sr1[f]) - (self.K[f]*self.sr1[f]*self.sr2[f])
            self.power[f] += samp*samp
            
            if (self.power[f] == 0):
                self.power[f] = 1
            #Calculate the actual frequencies for us to measure
            freqs[f] = power / self.power[f] / self.N[f]
            
        return self.number(freqs)
    
    

In [3]:
if __name__ == '__main__':
    
    
    #read wave files
    wave0 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_0.wav', 'r')
    wave1 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_1.wav', 'r')
    wave2 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_2.wav', 'r')
    wave3 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_3.wav', 'r')
    wave4 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_4.wav', 'r')
    wave5 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_5.wav', 'r')
    wave6 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_6.wav', 'r')
    wave7 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_7.wav', 'r')
    wave8 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_8.wav', 'r')
    wave9 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_9.wav', 'r')
    wavepound = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_pound.wav', 'r')
    wavestar = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_star.wav', 'r')
    mystery = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/mystery.wav', 'r')

    # load wav file from the above directory
    wav = mystery
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()
    frames = wav.readframes(nframes * nchannels)
    # convert wave file to array of integers
    frames = struct.unpack_from("%dH" % nframes * nchannels, frames)
    #set left/right channels equal to amount of total frames
    left = frames
    right = left
    #Adjust the binsize and split to mess with the noise in the sounds and try to get the desired outcome
    binsize = 205
    binsize_split = 10
    prevvalue = ""
    prevcounter = 0
    for i in range(0,len(left)-binsize,binsize//binsize_split):
        goertzel = dtmfdecoder(framerate)
        for j in left[i:i+binsize]:
            value = goertzel.calc(j)
        if value==prevvalue:
            prevcounter+=1
            if prevcounter==10:
                print(value)
        else:
            prevcounter=0
            prevvalue=value

8
6
7
2
5
3
0
9


# Part 2

In [4]:
#Class for DTMF Decoder
class robustdtmfdecoder:
    #Create Goertzel Parameters
    def __init__(self, sampr):
        self.sampr = sampr
        #Establish known frequencies that are given to us
        self.g_fr = [1209,1336,1477,1633,697,770,852,941]
        #Setup individual parameters so we can sample the file all at once
        self.sr1 = {}
        self.sr2 = {}
        self.power = {}
        self.N = {}
        self.K = {}
        self.sr1_h = {}
        self.sr2_h = {}
        self.power_h = {}
        self.N_h = {}
        self.K_h = {}
        
        for i in self.g_fr:
            self.sr1[i] = 0
            self.sr1_h[i] = 0
            self.sr2[i] = 0
            self.sr2_h[i] = 0
            self.power[i] = 0
            self.power_h[i] = 0
            self.N[i] = 0
            self.N_h[i] = 0
            normfreq = i/sampr
            self.K[i] = 2*np.cos(2*np.pi*normfreq)
            self.K_h[i] = 2*np.cos(2*np.pi*normfreq*2)
            
            
            
    #Program to generate the number
    def number(self, freq,freq_harm):
        
        #iterated to make the code robust for part 2 of the problem
        side = [697,770,852,941]
        bottom = [1209,1336,1477,1633]

        
        #get side freq
        sidefreq = 0
        sidefreq_v = 0
        #compare second and first harmonics
        for i in side:
            #compare the freq of the original and new harmonic
            if freq[i] > sidefreq_v and freq[i]/freq_harm[i]>1:
                 
                sidefreq_v = freq[i]
                sidefreq = i
        
                
        #get bottom freq
        botfreq = 0
        botfreq_v = 0
        botfreq_v2 = 0
        for i in bottom:
            #compare the freq of the original and new harmonic
            if freq[i] > botfreq_v and freq[i]/freq_harm[i]>1:
                   
                botfreq_v = freq[i]
                botfreq = i
        
        #assign number to frequecy going across the grids one by one
        if sidefreq==697.0:
            if botfreq==1209.0:
                return "1"
            elif botfreq==1336.0:
                return "2"
            elif botfreq==1477.0:
                return "3"
            elif botfreq==1633.0:
                return "A"
        elif sidefreq==770.0:
            if botfreq==1209.0:
                return "4"
            elif botfreq==1336.0:
                return "5"
            elif botfreq==1477.0:
                return "6"
            elif botfreq==1633.0:
                return "B"
        elif sidefreq==852.0:
            if botfreq==1209.0:
                return "7"
            elif botfreq==1336.0:
                return "8"
            elif botfreq==1477.0:
                return "9"
            elif botfreq==1633.0:
                return "C"
        elif sidefreq==941.0:
            if botfreq==1209.0:
                return "*"
            elif botfreq==1336.0:
                return "0"
            elif botfreq==1477.0:
                return "#"
            elif botfreq==1633.0:
                return "D"
        else:
            return 0
    #Run the calculation for the DTMF
    def calc(self,samp):
        freqs = {}
        freq_harm = {}
        for f in self.g_fr:
            s = samp + (self.K[f] * self.sr1[f]) - self.sr2[f]
            self.sr2[f] = self.sr1[f]
            self.sr1[f] = s
            self.N[f] += 1
            power = (self.sr2[f]*self.sr2[f]) + (self.sr1[f]*self.sr1[f]) - (self.K[f]*self.sr1[f]*self.sr2[f])
            self.power[f] += samp*samp
            
            if (self.power[f] == 0):
                self.power[f] = 1
            #Calculate the actual frequencies for us to measure
            freqs[f] = power / self.power[f] / self.N[f]
            
            #iterate for new harmonic frequency
            s_h = samp + (self.K_h[f] * self.sr1_h[f]) - self.sr2_h[f]
            self.sr2_h[f] = self.sr1_h[f]
            self.sr1_h[f] = s_h
            self.N_h[f] += 1
            power = (self.sr2_h[f]*self.sr2_h[f]) + (self.sr1_h[f]*self.sr1_h[f]) - (self.K_h[f]*self.sr1_h[f]*self.sr2_h[f])
            self.power_h[f] += samp*samp
            
            if (self.power_h[f] == 0):
                self.power_h[f] = 1
            #Calculate the actual frequencies for us to measure
            freq_harm[f] = power / self.power_h[f] / self.N_h[f]
            
            

            
        return self.number(freqs,freq_harm)

In [5]:
if __name__ == '__main__':
    
    
    #read wave files
    wave0 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_0.wav', 'r')
    wave1 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_1.wav', 'r')
    wave2 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_2.wav', 'r')
    wave3 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_3.wav', 'r')
    wave4 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_4.wav', 'r')
    wave5 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_5.wav', 'r')
    wave6 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_6.wav', 'r')
    wave7 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_7.wav', 'r')
    wave8 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_8.wav', 'r')
    wave9 = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_9.wav', 'r')
    wavepound = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_pound.wav', 'r')
    wavestar = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/touchtone_star.wav', 'r')
    mystery = wave.open('/Users/jacobeverly/Documents/DSP/HW4/soundfiles/mystery.wav', 'r')

    # load wav file from the above directory
    wav = mystery
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()
    frames = wav.readframes(nframes * nchannels)
    # convert wave file to array of integers
    frames = struct.unpack_from("%dH" % nframes * nchannels, frames)
    #set left/right channels equal to amount of total frames
    left = frames
    right = left
    #Adjust the binsize and split to mess with the noise in the sounds and try to get the desired outcome
    binsize = 205
    binsize_split = 10
    prevvalue = ""
    prevcounter = 0
    for i in range(0,len(left)-binsize,binsize//binsize_split):
        goertzel = robustdtmfdecoder(framerate)
        for j in left[i:i+binsize]:
            value = goertzel.calc(j)
        if value==prevvalue:
            prevcounter+=1
            if prevcounter==10:
                print(value)
        else:
            prevcounter=0
            prevvalue=value

8
6
7
5
3
0
9


"867-5309/Jenny" is a 1981 song written by Alex Call and Jim Keller and performed by Tommy Tutone is the pop culture reference being made